# Data Cleaning

## Text Extraction

In [1]:
%pip install pandas requests beautifulsoup4 openpyxl

Note: you may need to restart the kernel to use updated packages.


In [9]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import time
import re

# load the Excel file with URLs
df = pd.read_excel('web_scraping_urls.xlsx')

# function to clean text
def clean_text(text):
    # regex to match non-printable characters
    clean_text = re.sub(r'[\x00-\x1f\x7f-\x9f]', '', text)
    return clean_text

# function to extract and clean text from a URL
def extract_text(url):
    try:
        response = requests.get(url)
        response.raise_for_status()
        soup = BeautifulSoup(response.text, 'html.parser')
        text = soup.get_text()
        # clean the text before returning
        return clean_text(text)
    except requests.RequestException as e:
        return str(e)

# define batch size and output file name
batch_size = 10
output_filename = 'all_extracted_texts.xlsx'

# initialize the ExcelWriter to append without overwriting previous data
with pd.ExcelWriter(output_filename, engine='openpyxl', mode='a', if_sheet_exists='overlay') as writer:
    total_urls = len(df['URLs'])

    # process each batch
    for start in range(810, total_urls, batch_size):
    # start = 375
    # end = 380
        end = start + batch_size
        print(f"processing batch from URL {start+1} to {min(end, total_urls)}")
        batch_data = {'URL': df['URLs'][start:end], 'Quarter': df['Quarter'][start:end], 'Extracted Text': []}

        for index, url in enumerate(df['URLs'][start:end], start=start+1):
            print(f"processing URL {index}/{total_urls}: {url}")
            text = extract_text(url)
            batch_data['Extracted Text'].append(text)
            time.sleep(2)  # sleep for 2 seconds between requests
            print("extraction successful for this URL")

        # convert batch data to DataFrame and append to the Excel file
        batch_df = pd.DataFrame(batch_data)
        # append batch data to the Excel file
        batch_df.to_excel(writer, sheet_name='Extracted Text', index=False, header=not writer.sheets, startrow=writer.sheets['Extracted Text'].max_row if 'Extracted Text' in writer.sheets else 0)

        print(f"batch from URL {start+1} to {min(end, total_urls)} saved.")

print("all URLs processed and saved to the same Excel file.")


processing batch from URL 561 to 570
processing URL 561/4913: https://www.commercialguru.com.sg/listing/for-sale-kb-industrial-building-14443734
extraction successful for this URL
processing URL 562/4913: https://www.spglobal.com/marketintelligence/en/mi/research-analysis/asean-manufacturing-rebounds-in-october-rcep-ratification-boosts-longerterm-economic-outlook-nov21.html
extraction successful for this URL
processing URL 563/4913: https://www.sp.edu.sg/pace/news/pace/industrial-transformation-asia-pacific-(itap)-2021
extraction successful for this URL
processing URL 564/4913: https://www.retalkasia.com/profiles/2021/10/18/savills-asia-pacific-welcomes-jack-harness-director-regional-industrial
extraction successful for this URL
processing URL 565/4913: https://www.nea.gov.sg/committee-of-supply-2022/integrated-sustainability-report-2020-2021/review-of-fy2020/ensuring-a-clean-and-sustainable-environment-for-singapore/sustainable-and-resource-efficient-singapore
extraction successful fo

KeyboardInterrupt: 